In [2]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [19]:
url = 'https://hh.ru'
route = '/search/vacancy'
params = {
  'text': 'Бортпроводник',
  'page': 0
}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

In [20]:
vacancies = []

while True:
  response = requests.get(url + route, params=params, headers=headers)
  if response.ok:
    dom = bs(response.text, 'html.parser')

    page_vacancies = dom.select('div.vacancy-serp-item')

    if not page_vacancies:
      break
    
    for page_vacancy in page_vacancies:
      vacancy_data = {
        'salary_from': None,
        'salary_to': None,
        'salary_currency': None
      }

      link_node = page_vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
      salary_node = page_vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})

      vacancy_data['title'] = link_node.text
      vacancy_data['url'] = link_node['href']
      vacancy_data['from'] = url

      if salary_node:
        salary_list = re.sub('\u202f', '', salary_node.text).split()
        vacancy_data['salary_currency'] = salary_list.pop()

        if salary_list[0] == 'от':
          vacancy_data['salary_from'] = salary_list[1]
          if 'до' in salary_list:
            vacancy_data['salary_to'] = salary_list[3]
        elif salary_list[0] == 'до':
          vacancy_data['salary_to'] = salary_list[1]
        else:
          vacancy_data['salary_from'] = salary_list[0]
          if '–' in salary_list:
            vacancy_data['salary_from'] = salary_list[0]
            vacancy_data['salary_to'] = salary_list[2]
          else:
            vacancy_data['salary_to'] = salary_list[0]

      vacancies.append(vacancy_data)

    print(f"Обработано вакансий на {params['page'] + 1} странице: {len(page_vacancies)}")
    params['page'] += 1
  else:
    break

Обработано вакансий на 1 странице: 20
Обработано вакансий на 2 странице: 20
Обработано вакансий на 3 странице: 20
Обработано вакансий на 4 странице: 20
Обработано вакансий на 5 странице: 20
Обработано вакансий на 6 странице: 2


In [21]:
data = pd.DataFrame(vacancies)
data.sample(10)

,salary_from,salary_to,salary_currency,title,url,from
98,50000,None,руб.,Посудомойщик / Посудомойщица,https://togliatti.hh.ru/vacancy/50299562?from=...,https://hh.ru
31,100000,None,руб.,Бортпроводник (г. Москва),https://togliatti.hh.ru/vacancy/50474014?from=...,https://hh.ru
56,52000,65000,руб.,"Стюард-официант на ""САПСАН"" Москва",https://togliatti.hh.ru/vacancy/44399009?from=...,https://hh.ru
83,35000,50000,руб.,Стюард (кухонный рабочий) в отель,https://togliatti.hh.ru/vacancy/50308916?from=...,https://hh.ru
95,35000,50000,руб.,Стюард/кухонный рабочий в отель (Сочи),https://togliatti.hh.ru/vacancy/50323500?from=...,https://hh.ru
4,170000,180000,руб.,Бортпроводник (Qatar Airways),https://togliatti.hh.ru/vacancy/50706342?from=...,https://hh.ru
62,24400,None,руб.,Уборшица(уборщик)/Стюард,https://togliatti.hh.ru/vacancy/43343777?from=...,https://hh.ru
50,None,None,None,Бортпроводник,https://togliatti.hh.ru/vacancy/50052626?from=...,https://hh.ru
57,None,None,None,Бортпроводник/Стажер,https://togliatti.hh.ru/vacancy/50356658?from=...,https://hh.ru
9,None,None,None,Бортпроводник,https://togliatti.hh.ru/vacancy/50111368?from=...,https://hh.ru


In [22]:
with open('hh_vacancies.json', 'w') as f:
    json.dump(vacancies, f)